In [ ]:
%load_ext autotime

import numpy as np
from biphase_gpt.nano_gpt import GPT, GPTConfig
from tqdm import tqdm

from fluo import Fluorescence1D, Fluorescence2D

In [ ]:
num_samples = int(1e4)
num_pix = 21
Phi_dim = num_pix // 2 + 1

# Dataset and Model Memory Sizes

In [ ]:
def print_1D_dataset_size():
    print('\n1D dataset and model sizes')
    float_size = 32
    target_size = num_pix  # full phase is 2*num_pix-1
    input_size = (Phi_dim - 1) ** 2

    inputs_data_size = input_size * float_size * num_samples
    targets_data_size = target_size * float_size * num_samples

    config = GPTConfig(
        in_seq_len=input_size, out_seq_len=target_size, n_layer=2, n_head=4, n_embd=64
    )
    print('Model size:', GPT(config).get_num_params() / 1e6, 'M parameters\r\n')

    print(
        f'Inputs for {num_pix} pixels with {num_samples} samples consume',
        inputs_data_size / 1e9,
        'GB\r\n',
    )
    print(
        f'Targets for {num_pix} pixels with {num_samples} samples consume',
        targets_data_size / 1e9,
        'GB\r\n',
    )

    print(
        f'Inputs for {num_pix} pixels with {num_samples} samples consume',
        inputs_data_size / 1e9,
        'GB',
    )
    print(
        f'Targets for {num_pix} pixels with {num_samples} samples consume',
        targets_data_size / 1e9,
        'GB',
    )


def print_2D_dataset_size(print_model_size=True):
    print('\n2D dataset and model sizes')
    float_size = 32
    target_size = (
        num_pix
    ) ** 2  # full phase is (2*num_pix-1)**2, we only need one quadrant
    input_size = (Phi_dim - 1) ** 4
    # We will only give inputs to one quadrant at a time
    # Inference will be run on at least two quadrants

    inputs_data_size = input_size * float_size * num_samples
    targets_data_size = target_size * float_size * num_samples

    if print_model_size:
        config = GPTConfig(
            in_seq_len=input_size,
            out_seq_len=target_size,
            n_layer=2,
            n_head=4,
            n_embd=64,
        )
        print('Model size:', GPT(config).get_num_params() / 1e6, 'M parameters\r\n')

    print(
        f'Inputs for {num_pix} pixels with {num_samples} samples consume',
        inputs_data_size / 1e9,
        'GB',
    )
    print(
        f'Targets for {num_pix} pixels with {num_samples} samples consume',
        targets_data_size / 1e9,
        'GB',
    )


print_1D_dataset_size()
print_2D_dataset_size()

# Randomizing 1D Phases

In [ ]:
print(f'\nGenerating {num_samples} samples...')
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix))
for i in tqdm(range(num_samples)):
    # Generate random phase
    phase = np.random.uniform(-np.pi, np.pi, num_pix)
    phase[0] = 0
    phase_samples[i, :] = phase

    # Compute Phi matrix
    Phi = Fluorescence1D.compute_Phi_from_phase(phase)
    Phi_samples[i, :, :] = Phi

In [ ]:
print('Mean', np.mean(Phi_samples))
print('StdDev', np.std(Phi_samples))
print('RMS', np.mean(4 * Phi_samples**2))

print('Mean', np.mean(phase_samples))
print('StdDev', np.std(phase_samples))
print('RMS', np.mean(4 * phase_samples**2))

This is quite fast, maybe for this round of pre-training I should be generating data on the fly?

# Randomizing 2D Phases

In [ ]:
print(f'\nGenerating {num_samples} samples...')
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix, num_pix))
for i in tqdm(range(num_samples)):
    # Generate random phase
    phase = np.random.uniform(-np.pi, np.pi, (num_pix, num_pix))
    phase[0, num_pix // 2 + 1] = 0
    phase_samples[i, :, :] = phase

    # Compute Phi matrix
    Phi = Fluorescence2D.compute_Phi_from_phase(phase)
    Phi_samples[i, :, :, :, :] = Phi
print(Phi_samples.shape)

# Randomizing 1D Coordinates

In [ ]:
print(f'\nGenerating {num_samples} samples...')
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix))
fluo = Fluorescence1D(kmax=3, num_pix=num_pix, num_atoms=4)
for i in tqdm(range(num_samples)):
    fluo.num_atoms = np.random.randint(3, 20)
    fluo.randomize_coords()
    phase_samples[i, :] = fluo.coh_phase
    Phi_samples[i, :, :] = np.arccos(
        fluo.cosPhi_from_phase()
    )  # returned value here is absPhi

In [ ]:
print('Mean', np.mean(Phi_samples))
print('StdDev', np.std(Phi_samples))
print('RMS', np.mean(4 * Phi_samples**2))

print('Mean', np.mean(phase_samples))
print('StdDev', np.std(phase_samples))
print('RMS', np.mean(4 * phase_samples**2))

This method is slower, because we are doing the same computation for the random phases above but adding a process that computes particular phases. If we wanted to generate inputs on-the-fly to save memory, we could just compute a bunch of these "special" phases and perform the last (fast) step to compute Phi in the DataLoader.

Additionally, we could probably make randomize_coords a little bit more efficient and check cProfile results. Probably not a lot to gain though.

In [ ]:
for i in tqdm(range(num_samples)):
    phase = phase_samples[i, :]
    # Compute Phi matrix
    Phi = Fluorescence1D.compute_Phi_from_phase(phase[num_pix - 1 :])
    Phi_samples[i, :, :] = Phi

Because we pre-computed the phases in this previous loop, we achieve the best iteration speed yet of nearly 134e3 it/s. So perhaps even in the case where we are sampling random phases and we want to perform on-the-fly computation of the inputs to reach very large model sizes in 2D, it would be beneficial to pre-compute the random phases.

# Randomizing 2D Coordinates

In [ ]:
print(f'\nGenerating {num_samples} samples...')
Phi_samples = np.zeros((num_samples, Phi_dim, Phi_dim, Phi_dim, Phi_dim))
phase_samples = np.zeros((num_samples, num_pix, num_pix))
fluo = Fluorescence2D(kmax=3, num_pix=num_pix, num_atoms=4)
for i in tqdm(range(num_samples)):
    fluo.num_atoms = np.random.randint(3, 20)
    fluo.randomize_coords()
    phase_samples[i, :, :] = fluo.coh_phase
    Phi_samples[i, :, :, :, :] = np.arccos(
        fluo.cosPhi_from_phase()
    )  # returned value here is absPhi

In [ ]:
for i in tqdm(range(num_samples)):
    phase = phase_samples[i, :, :]
    # Compute Phi matrix
    Phi = Fluorescence2D.compute_Phi_from_phase(phase[num_pix - 1 :, num_pix - 1 :])
    Phi_samples[i, :, :, :, :] = Phi
print(Phi_samples.shape)

# Computing $g^{(3)}$

In [ ]:
num_shots = 1000

In [ ]:
fluo = Fluorescence1D(kmax=3, num_pix=num_pix, num_atoms=4)
for i in tqdm(range(num_samples)):
    fluo.num_atoms = np.random.randint(3, 20)
    fluo.randomize_coords()
    g3 = fluo.marginalize_g3(num_shots=num_shots)
    # We only need a small slice of this, can we avoid generating the whole 2*num_pix by 2*num_pix array?

If we are memory limited in how many g3 we can generate (if we have to pass through the 3D/6D array) or store (large datasets going as detector area squared), how efficient is it to generate this final round of fine-tuning data on-the-fly?

# $g^{(3)}$ Discretization Errors

How accurate is encoding when kmax is large and num_pix is small?